In [24]:
# 4-1. 프로젝트: 뉴스기사 요약해보기
# 새로운 데이터셋에 대해서 추상적 요약과 추출적 요약을 모두 해보는 시간을 가져봐요.

# 먼저 주요 라이브러리 버전을 확인해 보죠.

from importlib.metadata import version
import nltk
import tensorflow
import summa
import pandas as pd  
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords


# nltk 불용어 다운로드
nltk.download('stopwords')


print(nltk.__version__)
print(tensorflow.__version__)
print(pandas.__version__)
print(version('summa'))

3.6.5
2.6.0
1.3.3
1.2.0


[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Step 1. 데이터 수집하기
# 데이터는 아래 링크에 있는 뉴스 기사 데이터(news_summary_more.csv)를 사용하세요.

# sunnysai12345/News_Summary
# 아래의 코드로 데이터를 다운로드할 수 있어요.

import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [20]:
data.sample(10)

,headlines,text
9964,Nazar utar lo: Karan Johar wishes Deepika-Ranv...,Director Karan Johar took to Twitter to wish D...
72753,Your tears sear our heart: J&K Police to marty...,After pictures of slain J&K Police officer Abd...
908,Kangana aur main kisi ke aage peeche nahi ghum...,Actress Ankita Lokhande has said she has been ...
44627,Andhra CM tells PM Modi why he asked TDP minis...,Andhra Pradesh CM Chandrababu Naidu on Thursda...
54347,Welcome brother: Kamal on Rajinikanth's entry ...,Congratulating Rajinikanth on his entry into p...
12154,Interim CBI chief can't take any policy decisi...,During a Supreme Court hearing on government's...
67891,Tesla is 'structurally unprofitable': Investor...,American investor Jim Chanos has said that ele...
47573,Govt delayed Paytm UPI so Google could launch ...,Paytm Founder Vijay Shekhar Sharma has claimed...
40961,Israelis have right to their own land: Saudi C...,Saudi Arabia's Crown Prince Mohammed bin Salma...
8789,"I feel dismay, resentment over Carlos Ghosn sc...","In a letter to company employees, Japanese car..."


In [ ]:
# Step 2. 데이터 전처리하기 (추상적 요약)

# 실습에서 사용된 전처리를 참고하여 각자 필요하다고 생각하는 전처리를 추가 사용하여 텍스트를 정규화 또는 정제해 보세요. 
# 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해 보세요.

In [56]:
# 정규화 사전 및 불용어 로드

contractions = {"ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because", "could've": "could have",
               "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
               "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will", "he's": "he is",
               "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would",
               "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have",
               "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",
               "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will",
               "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
               "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
               "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
               "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
               "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
               "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",
               "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
               "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
               "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
               "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have",
               "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
               "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
               "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
               "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
               "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
               "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
               "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
               "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
               "y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have",
               "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}


In [58]:

# 데이터 전처리 함수
def preprocess_sentence(sentence, remove_stopwords=True):
    sentence = sentence.lower()                                 # 소문자 변환
    sentence = BeautifulSoup(sentence, "lxml").text             # HTML 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence)               # 괄호 안의 내용 제거
    sentence = re.sub('"', '', sentence)                        # 쌍따옴표 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")])  # 축약어 전체 단어로 변환
    sentence = re.sub(r"'s\b", "", sentence)                    # 소유격 제거
    sentence = re.sub("[^a-zA-Z]", " ", sentence)               # 영문자가 아닌 문자 공백으로 변환
    sentence = re.sub('[m]{2,}', 'mm', sentence)                # 연속된 문자 'm'을 'mm'으로 변경
    
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))            # NLTK에서 제공하는 불용어 다운로드
        
        # 불용어 제거 및 단어 길이가 1 이상인 단어만 선택
        tokens = ' '.join(word for word in sentence.split() if word not in stop_words and len(word) > 1) 
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)       # 단어 길이가 1 이상인 단어만 선택
    
    return tokens



# 전체 Summary 데이터에 대한 전처리
clean_summary = []

for idx, row in data.iterrows():
    cleaned_summary = preprocess_summary(row['headlines'])
    clean_summary.append(cleaned_summary)

# 전처리 후 출력
print("Summary 전처리 후 결과: ", clean_summary[:5])


Summary 전처리 후 결과:  ['upgrad learner switches career ml al salary hike', 'delhi techie wins free food swiggy one year cred', 'new zealand end rohit sharma led india match winning streak', 'aegon life iterm insurance plan helps customers save tax', 'known hirani yrs metoo claims true sonam']


In [ ]:
# Step 3. 어텐션 메커니즘 사용하기 (추상적 요약)
# 일반적인 seq2seq보다는 어텐션 메커니즘을 사용한 seq2seq를 사용하는 것이 더 나은 성능을 얻을 수 있어요. 
# 실습 내용을 참고하여 어텐션 메커니즘을 사용한 seq2seq를 설계해 보세요.

In [67]:
# 필요한 라이브러리 및 모듈 불러오기

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer

# 데이터 로드
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')


# 전체 Text 데이터에 대한 전처리
clean_text = []

for idx, row in data.iterrows():
    cleaned_text = preprocess_sentence(row['text'])
    clean_text.append(cleaned_text)

# 정수 인코딩을 위한 토크나이저 정의
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_text)

# 단어 집합 크기 설정
x_voc_size = len(tokenizer.word_index) + 1  # 1을 더해주는 이유는 패딩을 위한 0이 추가되기 때문입니다.

# 가장 긴 문장의 길이 계산
max_text_len = max(len(sentence.split()) for sentence in clean_text)

# 어텐션 메커니즘 사용한 seq2seq 모델 정의
latent_dim = 300

# 인코더
encoder_inputs = Input(shape=(max_text_len,))
enc_emb = Embedding(x_voc_size, latent_dim, trainable=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)


# 어텐션 층
attn_layer = Attention(name='attention_layer')
# 주의: decoder_outputs의 일부를 사용 (출력의 일부를 선택)
attn_out = attn_layer([encoder_outputs, decoder_outputs[:, :max_text_len, :]])  # Adjusted decoder_outputs dimensions

# 디코더의 출력과 어텐션의 결과를 연결
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

# Dense layer를 통해 출력 단어 예측
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


ValueError: Dimensions must be equal, but are 300 and 32402 for '{{node attention_layer/MatMul}} = BatchMatMulV2[T=DT_FLOAT, adj_x=false, adj_y=true](Placeholder, Placeholder_1)' with input shapes: [?,60,300], [?,13,32402].

In [ ]:
# Step 5. Summa을 이용해서 추출적 요약해보기
# 추상적 요약은 추출적 요약과는 달리 문장의 표현력을 다양하게 가져갈 수 있지만, 
# 추출적 요약에 비해서 난이도가 높아요. 반대로 말하면 추출적 요약은 추상적 요약에 비해 난이도가 낮고 기존 문장에서 문장을 꺼내오는 것이므로 잘못된 요약이 나올 가능성이 낮아요.

# Summa의 summarize를 사용하여 추출적 요약을 해보세요.

In [ ]:
'''
루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

평가문항	상세기준

1. Abstractive 모델 구성을 위한 텍스트 전처리 단계가 체계적으로 진행되었다.	분석단계, 정제단계, 정규화와 불용어 제거, 
데이터셋 분리, 인코딩 과정이 빠짐없이 체계적으로 진행되었다.


2. 텍스트 요약모델이 성공적으로 학습되었음을 확인하였다
.	모델 학습이 진행되면서 train loss와 validation loss가 감소하는 경향을 그래프를 통해 확인했으며, 
실제 요약문에 있는 핵심 단어들이 요약 문장 안에 포함되었다.


3. Extractive 요약을 시도해 보고 Abstractive 요약 결과과 함께 비교해 보았다.
두 요약 결과를 문법완성도 측면과 핵심단어 포함 측면으로 나누어 비교하고 분석 결과를 표로 정리하여 제시하였다.
'''

In [ ]:
'''

<프로젝트 보고서: 뉴스 기사 요약>

1. 개요

이 프로젝트는 뉴스 기사 데이터를 활용하여 추상적 요약과 추출적 요약을 수행하는 것을 목표로 합니다.
TensorFlow 및 Keras를 사용하여 Seq2Seq 모델을 구축하고 학습하고자 하였으나, 아직 코드가 완성되지 않아 성공하지 못한 상태입니다.

2. 데이터셋 로드 및 전처리

뉴스 기사 데이터는 'news_summary_more.csv'에서 가져왔으며, 데이터에는 원문인 'text'와 요약문인 'headlines'이 포함되어 있습니다. 
전처리는 HTML 태그 제거, 소문자 변환, 불용어 제거 등을 포함한 다양한 단계로 이루어졌습니다.

3. 모델 설계

아직 모델 설계 부분의 코드가 완성되지 않았습니다. 
하지만 기본적인 Seq2Seq 모델을 사용하고자 하였으며, 어텐션 메커니즘을 도입하여 더 나은 문장 요약을 기대하고 있습니다.

4. 성능 평가

현재는 모델 학습이 이루어지지 않았으므로 성능 평가 결과는 얻을 수 없었습니다.
모델 학습이 완료된 후에는 검증 데이터셋을 활용하여 성능을 평가할 예정입니다.

5. 모델 활용

모델이 성공적으로 학습되면, 새로운 뉴스 기사에 대한 추상적 요약 및 추출적 요약을 생성할 수 있을 것이라 예상합니다.
이를 통해 모델의 활용 가능성을 확인할 것입니다.


6. 결론

현재는 코드의 완성이 이루어지지 않아 전체적인 프로젝트 결과를 도출할 수 없었습니다.
그러나 프로젝트를 통해 뉴스 기사 요약에 대한 전반적인 이해와 Seq2Seq 모델의 구현에 대한 경험이 쌓았습니다.
향후에는 코드를 수정해서 프로젝트를 완성하고 뉴스 기사 요약을 생성할 수 있도록 노력해 보도록 하겠습니다.



<조필선 회고록>

죄송합니다. 노력은 하였지만 프로젝트를 완성하지 못하였습니다.

이번 프로젝트에서는 모델 구현이 미완성되어 아쉬움이 남지만, 데이터 전처리와 모델 설계 과정에서 딥러닝의 기본에 대한 
이해가 조금 더 높아진 것 같습니다.
코드의 성공 여부와는 별개로, 불용어 처리, 텍스트 정규화, 어텐션 메커니즘 등 다양한 기법을 적용하는 경험을 통해 
딥러닝에 대한 실전 스킬이 향상된 것 같은 느낌을 받았습니다.
비록 이번 프로젝트는 미완성되었지만 새로운 도전과 학습의 기회를 찾아낸 경험은 소중하다는 것을 느꼈습니다.
앞으로도 쉽지는 않겠지만 프로젝트에서 구현한 모델이 작동되도록 더 노력해 보겠습니다.
감사합니다.

'''